basically original 009.016 with quad_vec taking really long and I'm impatient so will submit 100 jobs for each ell_idx

In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
from LIMxCMBL.noise import *

import sys

Lambda_idx = 23#int(sys.argv[1])
nbins = 15#int(sys.argv[2])
curr_bin = 2#int(sys.argv[3])
Lambda = Lambdas[Lambda_idx]

zmin = 5.2#float(sys.argv[4])
zmax = 8#float(sys.argv[5])

line_str = 'Lya'#sys.argv[6]
ell_idx = 0#int(sys.argv[7])

chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))
chi_bin_edges = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), nbins + 1)
chi_bin_centers = (chi_bin_edges[1:] + chi_bin_edges[:-1])/2
dchi_binned = np.mean(np.diff(chi_bin_edges))


oup_fname = '/scratch/users/delon/LIMxCMBL/IHiKappa/'
oup_fname += '%s_IHik_zmin_%.1f_zmax_%.1f_idx_%d_dblquad_n_bins_%d_curr_%d_ell_idx_%d.npy'%(line_str,
                                                                                 zmin, 
                                                                                 zmax, 
                                                                                 Lambda_idx, 
                                                                                 nbins, 
                                                                                 curr_bin,
                                                                                           ell_idx)
print(oup_fname)

/scratch/users/delon/LIMxCMBL/IHiKappa/Lya_IHik_zmin_5.2_zmax_8.0_idx_23_dblquad_n_bins_15_curr_2_ell_idx_0.npy


In [2]:
# get CMB lensing component
from LIMxCMBL.kernels import get_f_Kkappa
f_WkD = get_f_Kkappa()

from LIMxCMBL.cross_spectrum import *
ClKK = d_chib_integral(f_WkD, f_WkD) #[Mpc]^2

kernels = {}
kernels['CII'] = np.array(KI)
kernels['CO'] = np.array(KI_CO)
kernels['Lya'] = np.array(KI_Lya)
kernels['HI'] = np.array(KI_HI)


_KI = kernels[line_str]


from  LIMxCMBL.kernels import *
f_KLIM   = interp1d(x=chis, y = _KI, bounds_error = False, fill_value=0)
f_Kkappa = get_f_Kkappa()

f_KLIM_windowed = apply_window(f_K = f_KLIM,
                               chimin = chimin,
                               chimax = chimax)

from scipy.interpolate import interp1d
from scipy.integrate import quad, quad_vec, trapezoid

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
inner_dkparp_integral = inner_dkparp_integral.astype(np.float64)



In [3]:
inner_dkparp_integral.shape

(100, 256, 128)

In [4]:
f_inner_integral = interp1d(x = chibs, y = inner_dkparp_integral[ell_idx], axis = 0)

from scipy.interpolate import interp1d, interp2d, LinearNDInterpolator

In [5]:
tmp_chibs = []
tmp_log_deltas = []
tmp_fnctn = []
for i in range(len(chibs)):
    for j in range(len(deltas)):
        tmp_chibs += [chibs[i]]
        tmp_log_deltas += [np.log(deltas[j])]
        tmp_fnctn += [inner_dkparp_integral[ell_idx,i,j]]
        
f_inner_integral_2d = LinearNDInterpolator(list(zip(tmp_chibs, tmp_log_deltas)), tmp_fnctn)

In [6]:
def f_KILo(chi, external_chi, Lambda):
    return (Lambda / np.pi 
            * np.interp(x = chi, xp = chis, 
                         fp = _KI, left = 0, right = 0) 
            * np.sinc(Lambda * (external_chi - chi) / np.pi))

In [7]:
def OLD_bin_integrand(_chib, chi):
    
    f_KLIMLo = get_f_KILo(external_chi = chi, Lambda=Lambda)
    f_KLIMLo_windowed = apply_window(f_K = f_KLIMLo,
                                     chimin = chimin,
                                     chimax = chimax)
    
    _curr_KI = 2 * np.interp(x = chi, xp = chis, fp = _KI, left = 0, right = 0)
    
    
    #inner integrand
    #Low passed
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)

    _interm  = f_KLIMLo_windowed(plus) * f_Kkappa(mins)
    _interm += f_KLIMLo_windowed(mins) * f_Kkappa(plus) 

    _factor = (2 / _chib)
    _factor = _factor * deltas
    _factor *= f_inner_integral(_chib)

    _interm  *= _factor

    LO_integrand = trapezoid(x = np.log(deltas), y = _interm, axis=-1)

    #unfiltered
    _delta = np.abs(1 - chi / _chib)
    _delta = np.where(_delta < 1e-6, 1e-6,
                     np.where(_delta > 0.7, 
                             0.7,
                             _delta))
    unfiltered_integrand = (_curr_KI 
                            * np.interp(x = 2*_chib - chi, 
                                        xp = chis, fp = Wk * Dz, 
                                        left = 0, right = 0)
                            * f_inner_integral_2d((_chib, np.log(_delta))) 
                            / _chib**2)

    return unfiltered_integrand - LO_integrand

In [8]:
def bin_integrand(_chib, chi):
    
    _curr_KI = 2 * np.interp(x = chi, xp = chis, fp = _KI, left = 0, right = 0)
    
    
    #inner integrand
    #Low passed
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)

    _interm  = np.where((plus >= chimin) & (plus <= chimax),
                        f_KILo(plus, external_chi = chi, Lambda=Lambda) 
                        * np.interp(x = mins, xp = chis, fp = Wk * Dz, left = 0, right = 0),
                        0)
    _interm += np.where((mins >= chimin) & (mins <= chimax),
                        f_KILo(mins, external_chi = chi, Lambda=Lambda) 
                        * np.interp(x = plus, xp = chis, fp = Wk * Dz, left = 0, right = 0),
                        0)
    _factor = (2 / _chib)
    _factor = _factor * deltas
    _factor *= f_inner_integral(_chib)

    _interm  *= _factor

    LO_integrand = trapezoid(x = np.log(deltas), y = _interm, axis=-1)

    #unfiltered
    _delta = np.abs(1 - chi / _chib)
    _delta = np.where(_delta < 1e-6, 1e-6,
                     np.where(_delta > 0.7, 
                             0.7,
                             _delta))
    unfiltered_integrand = (_curr_KI 
                            * np.interp(x = 2*_chib - chi, 
                                        xp = chis, fp = Wk * Dz, 
                                        left = 0, right = 0)
                            * f_inner_integral_2d((_chib, np.log(_delta))) 
                            / _chib**2)

    return unfiltered_integrand - LO_integrand

In [9]:
bin_integrand((chimin+chimax)/2, (chimin+chimax)/2)

1.7203217206309548e-13

In [10]:
OLD_bin_integrand((chimin+chimax)/2, (chimin+chimax)/2)

-3.445311882808094e-13

In [11]:
import cProfile

In [12]:
cProfile.run('[OLD_bin_integrand((chimin+chimax)/2, (chimin+chimax)/2) for _ in range(1000)]')

         340004 function calls in 0.398 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.060    0.000    0.396    0.000 350217941.py:1(OLD_bin_integrand)
        1    0.002    0.002    0.398    0.398 <string>:1(<listcomp>)
        1    0.000    0.000    0.398    0.398 <string>:1(<module>)
     7000    0.001    0.000    0.002    0.000 _base.py:1513(issparse)
     2000    0.001    0.000    0.001    0.000 _interpolate.py:381(_do_extrapolate)
     1000    0.018    0.000    0.078    0.000 _interpolate.py:496(__init__)
     1000    0.001    0.000    0.001    0.000 _interpolate.py:649(fill_value)
     1000    0.000    0.000    0.000    0.000 _interpolate.py:680(_check_and_update_bounds_error_for_extrapolation)
     2000    0.001    0.000    0.012    0.000 _interpolate.py:686(_call_linear_np)
     5000    0.052    0.000    0.087    0.000 _interpolate.py:690(_call_linear)
     7000    0.012    0.000    0.126    0.000 _

In [13]:
cProfile.run('[bin_integrand((chimin+chimax)/2, (chimin+chimax)/2) for _ in range(1000)]')

         124004 function calls in 0.175 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.066    0.000    0.174    0.000 1352390920.py:1(bin_integrand)
     2000    0.012    0.000    0.035    0.000 1612985584.py:1(f_KILo)
        1    0.001    0.001    0.175    0.175 <string>:1(<listcomp>)
        1    0.000    0.000    0.175    0.175 <string>:1(<module>)
     1000    0.000    0.000    0.000    0.000 _base.py:1513(issparse)
     1000    0.012    0.000    0.018    0.000 _interpolate.py:690(_call_linear)
     1000    0.004    0.000    0.030    0.000 _interpolate.py:759(_evaluate)
     1000    0.003    0.000    0.008    0.000 _interpolate.py:774(_check_bounds)
     1000    0.000    0.000    0.002    0.000 _methods.py:47(_sum)
     2000    0.001    0.000    0.004    0.000 _methods.py:55(_any)
     1000    0.002    0.000    0.002    0.000 _methods.py:90(_clip)
     1000    0.001    0.000    0.039    0.000 _polyint.p

In [14]:
from scipy.integrate import nquad
left = chi_bin_edges[curr_bin]
right = chi_bin_edges[curr_bin + 1]

options={'limit':1000, 'epsabs':0.0, 'epsrel':1e-3}

res, _ = nquad(bin_integrand, [[10,chimax_sample],[left, right]],
               opts=[options,options])

res /= dchi_binned

# np.save(oup_fname, res)
# print('outputted')

/home/users/delon/.local/lib/python3.9/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


In [15]:
_oup_fname = '/scratch/users/delon/LIMxCMBL/IHiKappa/'
_oup_fname += '%s_IHik_zmin_%.1f_zmax_%.1f_idx_%d_dblquad_n_bins_%d_curr_%d.npy'%(line_str,
                                                                                 zmin, 
                                                                                 zmax, 
                                                                                 Lambda_idx, 
                                                                                 nbins, 
                                                                                 curr_bin)
dbl_chk = np.load(_oup_fname)

In [16]:
print(res)

7.563205921477704e-15


In [17]:
print(dbl_chk[ell_idx])

7.570638757706707e-15
